# Named Entity Recognition using Language Models

This lab will focus on helping you fine-tune Transformers models for Named Entity Recognition. You will use the Datasets library to quickly load and preprocess the WikiNEuRal dataset, getting them ready for training with PyTorch.

In [ ]:
from IPython.display import HTML, display
colab_button = HTML(
    '<a target="_blank" href="https://colab.research.google.com/github/surrey-nlp/NLP-2025/blob/main/lab06/lab06-NER_using_LM.ipynb">'
    '<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'
)
display(colab_button)

## Installation
Let us first install the required libraries

In [1]:
!pip3 install datasets transformers seqeval huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=bc140315b08ea688974e46b76c260d399fe027a4dc5c3dd27feded3c39e0d21c
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.13
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
Need to get 8,489 kB of archives.
After this operation, 7,671 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu jammy/main amd64 git-lfs amd64 3.6.1 [8,489 kB]
Fetched 8,489 kB in 0s (30.7 MB/s)
debconf: unable to initialize frontend: Dialog
debco

## Dataset(s)

We will use the Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be done with the functions load_dataset and load_metric.

The notebook should work with any token classification dataset provided by the Datasets library.

In [4]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset, concatenate_datasets
from evaluate import load
datasets = load_dataset("Babelscape/wikineural")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.27k [00:00<?, ?B/s]

test_de-00000-of-00001.parquet:   0%|          | 0.00/847k [00:00<?, ?B/s]

test_en-00000-of-00001.parquet:   0%|          | 0.00/894k [00:00<?, ?B/s]

test_es-00000-of-00001.parquet:   0%|          | 0.00/723k [00:00<?, ?B/s]

...

Generating val_it split:   0%|          | 0/11050 [00:00<?, ? examples/s]

Generating val_nl split:   0%|          | 0/10460 [00:00<?, ? examples/s]

Generating val_pl split:   0%|          | 0/13520 [00:00<?, ? examples/s]

Generating val_pt split:   0%|          | 0/10070 [00:00<?, ? examples/s]

Generating val_ru split:   0%|          | 0/11540 [00:00<?, ? examples/s]

The WikiNEuRal dataset has train, validation and test splits for each of the 9 languages.

To access an actual element, you need to select a split first, then give an index:

In [7]:
datasets["train_en"][0]

{'tokens': ['This',
  'division',
  'also',
  'contains',
  'the',
  'Ventana',
  'Wilderness',
  ',',
  'home',
  'to',
  'the',
  'California',
  'condor',
  '.'],
 'ner_tags': [0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 5, 0, 0],
 'lang': 'en'}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored the following:

In [8]:
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

labels_vocab = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
labels_vocab_reverse = {v:k for k,v in labels_vocab.items()}

If you want to train a monolingual model (NER on a single language), you have to use a single language for each split.

In [9]:
train_dataset = datasets["train_en"]
val_dataset = datasets["val_en"]
test_dataset = datasets["test_en"]

This is how you can concatenate multiple dataset splits to train a Multilingual NER model. However, let us focus on Monolingual NER for now. You can uncomment and use the following for other languages

In [10]:
# train_dataset = concatenate_datasets([datasets["train_de"],
#                                       datasets["train_en"],
#                                       datasets["train_es"],
#                                       datasets["train_fr"],
#                                       datasets["train_it"],
#                                       datasets["train_nl"],
#                                       datasets["train_pl"],
#                                       datasets["train_pt"],
#                                       datasets["train_ru"]])

# val_dataset = concatenate_datasets([datasets["val_de"],
#                                     datasets["val_en"],
#                                     datasets["val_es"],
#                                     datasets["val_fr"],
#                                     datasets["val_it"],
#                                     datasets["val_nl"],
#                                     datasets["val_pl"],
#                                     datasets["val_pt"],
#                                     datasets["val_ru"]])

# test_dataset = concatenate_datasets([datasets["test_de"],
#                                     datasets["test_en"],
#                                     datasets["test_es"],
#                                     datasets["test_fr"],
#                                     datasets["test_it"],
#                                     datasets["test_nl"],
#                                     datasets["test_pl"],
#                                     datasets["test_pt"],
#                                     datasets["test_ru"]])

## Pre-processing

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

 - we get a tokenizer that corresponds to the model architecture we want to use,
 - we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") ## We use the BERT tokenizer

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

# Challenge 01

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [13]:
# TODO: Add assertion to ensure that our tokenizer is of `PreTrainedTokenizerFast` type.

import transformers
assert isinstance(...)

You can directly call this tokenizer on one sentence, or you can use the tokenize method to tokenize multiple sentences at once.

In [14]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

This is a good overal tutorial for preprocessing for Transformer. Check this out to learn more about the preprocessing steps.
https://huggingface.co/transformers/preprocessing.html

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. Just change the value of the following flag:

In [15]:
label_all_tokens = False

We're now ready to write the function that will preprocess our samples. We feed them to the tokenizer with the argument truncation=True (to truncate texts that are bigger than the maximum size allowed by the model) and is_split_into_words=True (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [16]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Challenge 02

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [17]:
# TODO: Add `tokenize_and_align_labels` function to map function to preprocess the dataset in batch format.

train_tokenized = train_dataset.map(...)
val_tokenized = val_dataset.map(...)
test_tokenized = test_dataset.map(...)

Map:   0%|          | 0/92720 [00:00<?, ? examples/s]

Map:   0%|          | 0/11590 [00:00<?, ? examples/s]

Map:   0%|          | 0/11597 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the Datasets library to avoid spending time on this step the next time you run your notebook. The Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. Datasets warns you when it uses cached files, you can pass load_from_cache_file=False in the call to map to not use the cached files and force the preprocessing to be applied again.

Note that we passed batched=True to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning

In [18]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) and randomly initializing some other (the pre_classifier and classifier layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a Trainer, we will need to define three more things. The most important is the TrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [30]:
model_name = "bert-base-multilingual-cased" ## You Change this to any generic language model from HuggingFace which supports fine-tuning.
args = TrainingArguments(                   ## This is where all your hyperparameters are, reduce *_batch_size if you are working with a GPU which has limited memory.
    "wikineural-multilingual-ner",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=1,                     ## may not be enough for a large data, increase this to 5 later in your experiments.
    weight_decay=0.01,
    # push_to_hub=True,
    eval_steps=10000,
    save_steps=10000,
    report_to="none",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate,customize the batch_size and the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [20]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our Trainer is how to compute the metrics from the predictions. Here we will load the seqeval metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [21]:
metric = load("seqeval")

So we will need to do a bit of post-processing on our predictions:

 - select the predicted index (with the maximum logit) for each token
 - convert it to its string label
 - ignore everywhere we set a label of -100

The following function does all this post-processing on the result of Trainer.evaluate (which is a namedtuple containing predictions and labels) before applying the metric:

In [22]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Before you go any further:
1. Login to the HuggigFace CLI on your system using command ``huggingface-cli login``
2. Get your access token from your user account on HuggingFace at https://huggingface.co/settings/tokens (preferably a write access token)
3. Use this token with the command above to ensure being logged in to HuggingFace.

IF you are not working on your own machine with a GPU and have opened this notebook in Colab / Studio Lab, you can use the also cell below to login. You could have used this cell instead of commandline (cli) login, but where is the fun in that?

In [25]:
# from huggingface_hub import notebook_login
# notebook_login()

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the Trainer:

In [31]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-31-25887df61a95>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# Challenge 03

In [32]:
# TODO: Train the model

Step,Training Loss,Validation Loss


TrainOutput(global_step=2898, training_loss=0.047048578597990866, metrics={'train_runtime': 1146.5186, 'train_samples_per_second': 80.871, 'train_steps_per_second': 2.528, 'total_flos': 3357557343482976.0, 'train_loss': 0.047048578597990866, 'epoch': 1.0})

## Evaluation

The evaluate method allows you to evaluate again on the evaluation dataset or on another dataset:

# Challenge 04

In [33]:
# TODO: Evaluate the model

{'eval_loss': 0.027087325230240822,
 'eval_precision': 0.9211350095930526,
 'eval_recall': 0.9272681067344346,
 'eval_f1': 0.9241913831969808,
 'eval_accuracy': 0.9911624337931163,
 'eval_runtime': 44.7727,
 'eval_samples_per_second': 259.02,
 'eval_steps_per_second': 16.193,
 'epoch': 1.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the predict method:

In [34]:
predictions, labels, _ = trainer.predict(test_tokenized)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'LOC': {'precision': 0.9345810150059012,
  'recall': 0.930814441645676,
  'f1': 0.9326939256267878,
  'number': 5955},
 'MISC': {'precision': 0.8604070660522274,
  'recall': 0.8853981426595534,
  'f1': 0.8727237316194372,
  'number': 5061},
 'ORG': {'precision': 0.9321885913853318,
  'recall': 0.9286749782545666,
  'f1': 0.9304284676833696,
  'number': 3449},
 'PER': {'precision': 0.9590900401452878,
  'recall': 0.9629558541266795,
  'f1': 0.9610190594770616,
  'number': 5210},
 'overall_precision': 0.9211350095930526,
 'overall_recall': 0.9272681067344346,
 'overall_f1': 0.9241913831969808,
 'overall_accuracy': 0.9911624337931163}

Now, can you try to evaluate the NER task performance on a multilingual dataset? with a different language model?
1. Think about what Language Model on HuggingFace can help you Fine-tune over a multilingual dataset. Hint: XLM-R
2. Go back to the commented out multilingual dataset concatetation notebook cell and try to include a few more languages in the dataset.
3. Try to fine-tune the model on this dataset.
4. Report the performance in a MARKDOWN CELL below, showing your work on multilingual model fine-tuning in the cells below.